In [3]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt


In [66]:
pip install scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 4.4 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [4]:
df1 = pd.read_csv("/Users/jonathansantos/Desktop/IronHack/EDA-Inferential-stats/data/raw/df_final_demo.txt")

df2 = pd.read_csv("/Users/jonathansantos/Desktop/IronHack/EDA-Inferential-stats/data/raw/df_final_experiment_clients.txt")
df3 = pd.read_csv("/Users/jonathansantos/Desktop/IronHack/EDA-Inferential-stats/data/raw/df_final_web_data_pt_1.txt")
df4 = pd.read_csv("/Users/jonathansantos/Desktop/IronHack/EDA-Inferential-stats/data/raw/df_final_web_data_pt_2.txt")


In [5]:
# Concatenate df_final_web_data_pt_1 and df_final_web_data_pt_2 into a single DataFrame
df_final_web_data = pd.concat([df3, df4])

# Merge df_final_web_data with df_final_experiment_clients on 'client_id'
df_merged = pd.merge(df_final_web_data, df2[['client_id', 'Variation']], on='client_id', how='left')

In [6]:
# Define the columns to be added
columns_to_add = ['client_id', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr', 'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth']

# Merge df_merged with df_final_demo on 'client_id', adding the specified columns
df_merged = pd.merge(df_merged, df1[columns_to_add], on='client_id', how='left')

In [10]:
# Drop duplicates based on 'bal' column
df_merged_unique = df_merged.drop_duplicates(subset='bal')

# Sort and get top 100
top_100_clients = df_merged_unique.sort_values(by = "bal", ascending=False).head(100)

# Reset index and adjust it to start from 1
top_100_clients.reset_index(drop=True, inplace=True)
top_100_clients.index = top_100_clients.index + 1

print("Top 100 clients list")
for index, client in top_100_clients["client_id"].items():
    print(f"Rank: {index}, Client ID: {client}")

Top 100 clients list
Rank: 1, Client ID: 9286288
Rank: 2, Client ID: 8339240
Rank: 3, Client ID: 7070097
Rank: 4, Client ID: 9440656
Rank: 5, Client ID: 2291985
Rank: 6, Client ID: 1678193
Rank: 7, Client ID: 6762377
Rank: 8, Client ID: 3519657
Rank: 9, Client ID: 987970
Rank: 10, Client ID: 857273
Rank: 11, Client ID: 5939963
Rank: 12, Client ID: 7959723
Rank: 13, Client ID: 3559689
Rank: 14, Client ID: 9290030
Rank: 15, Client ID: 5735570
Rank: 16, Client ID: 3545814
Rank: 17, Client ID: 8056743
Rank: 18, Client ID: 2243576
Rank: 19, Client ID: 8700059
Rank: 20, Client ID: 6765532
Rank: 21, Client ID: 5799151
Rank: 22, Client ID: 8841326
Rank: 23, Client ID: 9848978
Rank: 24, Client ID: 8962152
Rank: 25, Client ID: 6981798
Rank: 26, Client ID: 7991760
Rank: 27, Client ID: 2935280
Rank: 28, Client ID: 1310232
Rank: 29, Client ID: 4179430
Rank: 30, Client ID: 9990452
Rank: 31, Client ID: 606032
Rank: 32, Client ID: 9578246
Rank: 33, Client ID: 6146018
Rank: 34, Client ID: 6271257
Rank:

In [11]:

# Once we have our top 100 clients depending on the balance amount, we proceed to check the ages of these clients.
# To do so, we'll take the average of the clnt_age column, and we consider the boundary between young and old in 40 years old.
average_clients_age = top_100_clients["clnt_age"].mean()
print(f"The average age of the top 100 clients is {average_clients_age} years.")

youngest_client_age = top_100_clients["clnt_age"].min()
print(f"The youngest client of the primary ones is {youngest_client_age} y/o.")

# Since we've seen that the youngest client is under the age we decided to consider them young or old, we count exactly how many clients of each type we have
young_clients, old_clients = (top_100_clients["clnt_age"] < 40).sum(), (top_100_clients["clnt_age"] >= 40).sum()
print(f"The total amount of young clients is {young_clients}, while there are {old_clients} of older clients out of the total 100 primary clients.")

# We do the same process for tenure years
average_tenure_years = top_100_clients["clnt_tenure_yr"].mean()
print(f"The average years of tenure is {average_tenure_years}.")
lowest_tenure_value = top_100_clients["clnt_tenure_yr"].min()
print(f"The most recent client out of the primary ones has been a client for {lowest_tenure_value} years.")

# We will consider 10 years as the minimun to say a client has been with the investing management company for a long time.
low_tenure_years, long_tenure_years = (top_100_clients["clnt_tenure_yr"] < 10).sum(), (top_100_clients["clnt_tenure_yr"] >= 10).sum()
print(f"There are {low_tenure_years} new clients, and {long_tenure_years} loyal clients.")


The average age of the top 100 clients is 58.645 years.
The youngest client of the primary ones is 26.5 y/o.
The total amount of young clients is 7, while there are 93 of older clients out of the total 100 primary clients.
The average years of tenure is 20.68.
The most recent client out of the primary ones has been a client for 4.0 years.
There are 14 new clients, and 86 loyal clients.


In [12]:
# Calculate the average balance for each gender
average_balance_by_gender = df_merged.groupby('gendr')['bal'].mean()
print(average_balance_by_gender)

gendr
F    143613.237247
M    234590.317485
U    100422.379386
X     27685.110714
Name: bal, dtype: float64


In [13]:
# Create a DataFrame where Variation = 'Test'
df_test = df_merged[df_merged['Variation'] == 'Test']
# Create a DataFrame where Variation = 'Control'
df_control = df_merged[df_merged['Variation'] == 'Control']

In [14]:
df_test.head(10)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test,22.0,274.0,34.5,M,2.0,36001.90,5.0,8.0
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43,Test,22.0,274.0,34.5,M,2.0,36001.90,5.0,8.0


In [15]:
# Convert the date and time column to datetime type since it is not yet in that format.
df_control['date_time'] = pd.to_datetime(df_control['date_time'])

# Sort the DataFrame by 'client_id' and 'date_time' to ensure that the records are in the correct order
df_control = df_control.sort_values(by=['client_id', 'date_time'])

# Calculate the time difference between each consecutive record for each user.
df_control['duration'] = (df_control['date_time'] - df_control.groupby('client_id')['date_time'].shift()).dt.total_seconds()

# Set the duration for the first record for each user, setting the time difference to 0
df_control.loc[df_control['client_id'] != df_control['client_id'].shift(), 'duration'] = 0

# Calculate the average time in each step.
average_time_per_step_control = df_control.groupby('process_step')['duration'].mean()

print("Average time in each step in minutes in Control:")
print((average_time_per_step_control/60).round(2))  # turn to minutes.


Average time in each step in minutes in Control:
process_step
confirm    1098.90
start      3115.16
step_1       30.55
step_2        0.66
step_3        9.58
Name: duration, dtype: float64


/var/folders/7p/ckhj31hn13n_2bj6qvx3j01c0000gn/T/ipykernel_1400/2118078490.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_control['date_time'] = pd.to_datetime(df_control['date_time'])


In [17]:
df_test.head(10)

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
5,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:17:15,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
6,9988021,580560515_7732621733,781255054_21935453173_531117,step_1,2017-04-17 15:17:01,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
7,9988021,580560515_7732621733,781255054_21935453173_531117,start,2017-04-17 15:16:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
8,8320017,39393514_33118319366,960651974_70596002104_312201,confirm,2017-04-05 13:10:05,Test,22.0,274.0,34.5,M,2.0,36001.90,5.0,8.0
9,8320017,39393514_33118319366,960651974_70596002104_312201,step_3,2017-04-05 13:09:43,Test,22.0,274.0,34.5,M,2.0,36001.90,5.0,8.0


In [18]:
# Calculate time spent in each state for Test
# Convert the date and time column to datetime type since it is not yet in that format.
df_test['date_time'] = pd.to_datetime(df_test['date_time'])
# Sort the DataFrame by 'client_id' and 'date_time' to ensure that the records are in the correct order
df_test = df_test.sort_values(by=['client_id', 'date_time'])
# Calculate the time difference between each consecutive record for each user.
df_test['duration'] = (df_test['date_time'] - df_test.groupby('client_id')['date_time'].shift()).dt.total_seconds()
# Set the duration for the first record for each user, setting the time difference to 0
df_test.loc[df_test['client_id'] != df_test['client_id'].shift(), 'duration'] = 0
# Calculate the average time in each step.
average_time_per_step_test = df_test.groupby('process_step')['duration'].mean()
print("Average time in each step in minutes in Test:")
print((average_time_per_step_test/60).round(2))# turn to minutes.

Average time in each step in minutes in Test:
process_step
confirm    1725.25
start      2376.64
step_1       22.28
step_2        1.85
step_3       12.30
Name: duration, dtype: float64


/var/folders/7p/ckhj31hn13n_2bj6qvx3j01c0000gn/T/ipykernel_1400/869945699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['date_time'] = pd.to_datetime(df_test['date_time'])


In [19]:
# Calculate time spent in each state for Control
# Convert the date and time column to datetime type since it is not yet in that format.
df_control['date_time'] = pd.to_datetime(df_control['date_time'])
# Sort the DataFrame by 'client_id' and 'date_time' to ensure that the records are in the correct order
df_control = df_control.sort_values(by=['client_id', 'date_time'])
# Calculate the time difference between each consecutive record for each user.
df_control['duration'] = (df_control['date_time'] - df_control.groupby('client_id')['date_time'].shift()).dt.total_seconds()
# Set the duration for the first record for each user, setting the time difference to 0
df_control.loc[df_control['client_id'] != df_control['client_id'].shift(), 'duration'] = 0
# Calculate the average time in each step.
average_time_per_step_control = df_control.groupby('process_step')['duration'].mean()
print("Average time in each step in minutes in Control:")
print((average_time_per_step_control/60).round(2))# turn to minutes.

Average time in each step in minutes in Control:
process_step
confirm    1098.90
start      3115.16
step_1       30.55
step_2        0.66
step_3        9.58
Name: duration, dtype: float64


In [41]:
 #Completion Rate of Test:
# Count the number of users who reached the 'confirm' step
n_completions = df_control[df_control['process_step'] == 'confirm']['client_id'].nunique()
# Count the total number of unique users
n_users = df_control['client_id'].nunique()
# Calculate the completion rate
completion_rate_control = n_completions / n_users * 100
# Print the completion rate
print("Completion rate of Control:", completion_rate_control)

Completion rate of Control: 65.58728539860616


In [21]:
# Completion Rate of Control
# Count the number of users who reached the 'confirm' step
n_completions = df_test[df_test['process_step'] == 'confirm']['client_id'].nunique()
# Count the total number of unique users
n_users = df_test['client_id'].nunique()
# Calculate the completion rate
completion_rate_test = n_completions / n_users * 100
# Print the completion rate
print("Completion rate of Test:", completion_rate_test)


Completion rate of Test: 69.29323642835953


In [22]:
from scipy.stats import ttest_ind

# Filter the dataframes for those clients who have reached the 'confirm' step.
cr_control = df_control[df_control['process_step'] == 'confirm']['client_id']
cr_test = df_test[df_test['process_step'] == 'confirm']['client_id']

# Use the two sample t-test to compare proportions.
_, p_value = ttest_ind(cr_control, cr_test)

print(f"The p value is: {p_value}")

alpha = 0.05  # Assuming alpha is 0.05
if p_value > alpha:
    print("The result is: We are not able to reject the null hypothesis")
else:
    print("The result is: We reject the null hypothesis")

The p value is: 0.04946102402798357
The result is: We reject the null hypothesis


In [23]:
# Statistics for the clnt_age, gendr, and bal columns from df_control and df_test
# For df_control
print("For df_control:")
# Age
print("\nClient Age:")
print("Median:", df_control['clnt_age'].median())
print("Mode:", df_control['clnt_age'].mode().values[0])
# Gender
print("\nGender:")
print("Percentage:", df_control['gendr'].value_counts(normalize=True) * 100)
# Balance
print("\nBalance:")
print("Average:", df_control['bal'].mean())
print("Median:", df_control['bal'].median())
# For df_test
print("\n\nFor df_test:")
# Age
print("\nClient Age:")
print("Median:", df_test['clnt_age'].median())
print("Mode:", df_test['clnt_age'].mode().values[0])
# Gender
print("\nGender:")
print("Percentage:", df_test['gendr'].value_counts(normalize=True) * 100)
# Balance
print("\nBalance:")
print("Average:", df_test['bal'].mean())
print("Median:", df_test['bal'].median())

For df_control:

Client Age:
Median: 49.5
Mode: 58.5

Gender:
Percentage: gendr
U    34.416399
M    33.693348
F    31.890252
Name: proportion, dtype: float64

Balance:
Average: 159398.63899895412
Median: 69009.64


For df_test:

Client Age:
Median: 50.5
Mode: 62.5

Gender:
Percentage: gendr
M    33.573883
U    33.480513
F    32.941104
X     0.004500
Name: proportion, dtype: float64

Balance:
Average: 164477.4268792994
Median: 69380.44


In [16]:
# Quartiles for the clnt_age and bal columns from df_control and df_test:
# For df_control
print("For df_control:")
# Age
print("\nClient Age:")
print("Quartiles:", df_control['clnt_age'].quantile([.25, .5, .75]))
# Balance
print("\nBalance:")
print("Quartiles:", df_control['bal'].quantile([.25, .5, .75]))
# For df_test
print("\n\nFor df_test:")
# Age
print("\nClient Age:")
print("Quartiles:", df_test['clnt_age'].quantile([.25, .5, .75]))
# Balance
print("\nBalance:")
print("Quartiles:", df_test['bal'].quantile([.25, .5, .75]))

For df_control:

Client Age:
Quartiles: 0.25    34.5
0.50    49.5
0.75    60.5
Name: clnt_age, dtype: float64

Balance:
Quartiles: 0.25     41129.12
0.50     69009.64
0.75    149784.99
Name: bal, dtype: float64


For df_test:

Client Age:
Quartiles: 0.25    35.0
0.50    50.5
0.75    61.0
Name: clnt_age, dtype: float64

Balance:
Quartiles: 0.25     41006.10
0.50     69380.44
0.75    152867.16
Name: bal, dtype: float64


In [22]:
#In order to obtain time spent in each step, we need to convert "date_time" column into date type.
df_merged["date_time"].dtypes

df_merged["date_time"] = pd.to_datetime(df_merged["date_time"])

#Once date_time is converted, we sort by id and date_time.
df1_sorted = df_merged.sort_values(by = ["client_id", "visit_id", "date_time"])

grouped = df1_sorted.groupby(["client_id", "visit_id"])

#We calculate the difference
df1_sorted["time_diff"] = grouped["date_time"].diff()

#We convert the values in seconds, so it will be clearer to understand
df1_sorted["time_diff"] = df1_sorted["time_diff"].dt.total_seconds()

#Now we can calculate the avg of time spent in each step
avg_time_step = df1_sorted.groupby(["Variation", "process_step"])["time_diff"].mean()
print(avg_time_step)

Variation  process_step
Control    confirm         128.664715
           start           151.366026
           step_1           42.998741
           step_2           38.667857
           step_3           92.944258
Test       confirm         128.769658
           start           147.140325
           step_1           37.704369
           step_2           48.131611
           step_3           96.850972
Name: time_diff, dtype: float64


In [25]:
df_merged.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0


In [32]:
 #Export df_control to a CSV file
df_control.to_csv('df_control.csv', index=False)
# Export df_control to a CSV file
df_test.to_csv('df_test.csv', index=False)

In [33]:

df_merged.to_csv('df_merged.csv', index=False)

In [42]:
# Calculate Error Rate for Test

# Convert 'date_time' to datetime if it's not already
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

# Sort the DataFrame by 'client_id' and 'date_time'
df_test = df_test.sort_values(['client_id', 'date_time'])

# Calculate the previous step for each client
df_test['previous_step'] = df_test.groupby('client_id')['process_step'].shift()

# Identify errors where the current step is less than the previous step
df_test['error'] = df_test['process_step'] <= df_test['previous_step']

# Calculate the total number of errors
total_errors = df_test['error'].sum()

# Calculate the total number of rows
total_rows = len(df_control)

# Calculate the error rate
error_rate = total_errors / total_rows

# Print the error rate in percentage
print("Error Rate for Test: {:.2%}".format(error_rate))

Error Rate for Test: 44.45%


In [43]:
# Calculate Error Rate for Control

# Convert 'date_time' to datetime if it's not already
df_control['date_time'] = pd.to_datetime(df_control['date_time'])

# Sort the DataFrame by 'client_id' and 'date_time'
df_control = df_control.sort_values(['client_id', 'date_time'])

# Calculate the previous step for each client
df_control['previous_step'] = df_control.groupby('client_id')['process_step'].shift()

# Identify errors where the current step is less than the previous step
df_control['error'] = df_control['process_step'] <= df_control['previous_step']

# Calculate the total number of errors
total_errors = df_control['error'].sum()

# Calculate the total number of rows
total_rows = len(df_control)

# Calculate the error rate
error_rate = total_errors / total_rows

# Print the error rate in percentage
print("Error Rate for Control: {:.2%}".format(error_rate))

Error Rate for Control: 33.42%


In [45]:

top_100_clients.to_csv('top_100_clients.csv', index=False)

In [46]:
# Query the DataFrame
result = df_merged.query("process_step in ['start', 'confirm']")

print(result['client_id'])

7         9988021
8         8320017
12        8320017
13        4033851
14        4033851
           ...   
755392    9668240
755400    9668240
755401    9668240
755403    9668240
755404     674799
Name: client_id, Length: 346908, dtype: int64
